# folium을 이용한 서울시 스타벅스 이디야 입지 시각화
- 참고 : https://wooiljeong.github.io/python/folium/

- 필요한 import 및 from 작업

In [ ]:
#!pip install folium

In [ ]:
import folium
import json
import glob
import os
import pandas as pd
import numpy as np

- 가장 최근 "소상공인시장진흥공단_상가(상권)정보 파일"을 다운로드합니다.
- 링크 : https://www.data.go.kr/data/15083033/fileData.do

- 데이터 불러오기

In [ ]:
#path = 'store202209/'
path = 'store202403/'
allFiles = glob.glob(os.path.join(path+"*.csv"))

df = pd.DataFrame()
for file in allFiles:

    print(file)

    temp = pd.read_csv(file, engine='python', encoding='utf-8')
    df = pd.concat([df, temp])

- 데이터 컬럼 확인하기

In [ ]:
print('컬럼 항목 수 :', len(df.columns))
list(df.columns)

- 상권업종 간단하게 확인하기

In [ ]:
print('='*70)
print('상권업종대분류명', set(df['상권업종대분류명']))
print('='*70)
print('상권업종중분류명', set(df['상권업종중분류명']))
print('='*70)

- 관련 정보만 추출

In [ ]:
dataset = df[['상호명','지점명',
              '상권업종대분류명', '상권업종중분류명',
              '시도명', '시군구명', '행정동명',
              '위도', '경도']]

dataset.head()

- 커피 전문점 데이터 추출
- 여러 상권업소 중 서울시 내 커피 전문점 업소만 추출해 df_coffee를 정의합니다.
- 즉, 시도명이 서울특별시이면서, 상권업종중분류명이 커피점/카페인 경우를 뜻합니다.

In [ ]:
sdata = dataset[dataset['상호명']=='스타벅스']
sdata

In [ ]:
if path == 'store202209/':
    sdata.to_csv("data/starbucks_raw_csv.csv", encoding='cp949')

In [ ]:
#df_coffee = dataset[(dataset['상권업종중분류명']=='커피점/카페')&(dataset['시도명']=='서울특별시')]
df_coffee = dataset[(dataset['상호명']=='스타벅스')&(dataset['시도명']=='서울특별시')]
df_coffee.index = range(len(df_coffee))
print('서울시 내 커피 전문점 점포 수 :', len(df_coffee))
df_coffee.head()

In [ ]:
df_coffee['상호명']

- 스타벅스 데이터 추출

In [ ]:
df_seoul_starbucks = df_coffee[df_coffee['상호명'].str.contains('스타벅스')]
df_seoul_starbucks.index = range(len(df_seoul_starbucks))
print('서울시 내 스타벅스 점포 수 :', len(df_seoul_starbucks))
df_seoul_starbucks.head()

- 이디야 데이터 추출

In [ ]:
df_seoul_ediya = df_coffee[df_coffee['상호명'].str.contains('이디야')]
df_seoul_ediya.index = range(len(df_seoul_ediya))
print('서울시 내 이디야 점포 수 :', len(df_seoul_ediya))
df_seoul_ediya.head()

- 메가커피 데이터 추출

In [ ]:
df_seoul_mega = df_coffee[df_coffee['상호명'].str.contains('메가커피')]
df_seoul_mega.index = range(len(df_seoul_mega))
print('서울시 내 메가커피 점포 수 :', len(df_seoul_mega))
df_seoul_mega.head()

- 서울시 구 별 스타벅스 수

In [ ]:
starbucks_gu = df_seoul_starbucks.groupby('시군구명')['상호명'].count().to_frame().sort_values(by='상호명', ascending=False)
starbucks_gu = starbucks_gu.reset_index()
starbucks_gu = starbucks_gu.set_index('시군구명')
starbucks_gu

- 서울시 구 별 이디야 수

In [ ]:
ediya_gu = df_seoul_ediya.groupby('시군구명')['상호명'].count().to_frame().sort_values(by='상호명', ascending=False)
ediya_gu = ediya_gu.reset_index()
ediya_gu = ediya_gu.set_index('시군구명')
ediya_gu

- 서울시 구 별 메가커피 수

In [ ]:
mega_gu = df_seoul_mega.groupby('시군구명')['상호명'].count().to_frame().sort_values(by='상호명', ascending=False)
mega_gu = mega_gu.reset_index()
mega_gu = mega_gu.set_index('시군구명')
mega_gu

- 서울특별시 폴리곤 JSON 파일 불러오기
- https://github.com/PinkWink/DataScience/blob/master/data/02.%20skorea_municipalities_geo_simple.json

folium을 통해 서울시 지도 위에 점포들을 시각화하기 위해서는 지리 정보를 담고 있는 폴리곤 json 파일이 필요합니다.
PinkWink(github)에 접속 후 02.%20skorea_municipalities_geo_simple.json을 map/ 폴더에 다운로드하여 불러옵니다.

간편 다운로드 방법

skorea_municipalities_geo_simple.json 링크를 우클릭하여 다른 이름으로 링크 저장 합니다.
(출처:PinkWink(github))

In [ ]:
geo_path = 'map/02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [ ]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)
data_size = len(df_seoul_starbucks)

# 지도 정의
map_starbucks = folium.Map(location=loc, zoom_start=12)
folium.Choropleth(geo_data=geo_str,
              data = starbucks_gu['상호명'],
              columns=[starbucks_gu.index, starbucks_gu['상호명']],
              fill_color='PuRd',
              key_on='feature.id')


# 포인트 마커 추가

for i in range(data_size):

    folium.Marker(list(df_seoul_starbucks.iloc[i][['위도', '경도']]),
                 popup=df_seoul_starbucks.iloc[i][['지점명']],
                 icon=folium.Icon(color='green')).add_to(map_starbucks)

map_starbucks

- 서울시 구 별 이디야 분포도 시각화

In [ ]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)
data2_size = len(df_seoul_ediya)

# 지도 정의
map_ediya = folium.Map(location=loc, zoom_start=12)
folium.Choropleth(geo_data=geo_str,
              data = ediya_gu['상호명'],
              columns=[ediya_gu.index, ediya_gu['상호명']],
              fill_color='PuRd',
              key_on='feature.id')


# 포인트 마커 추가

for i in range(data2_size):

    folium.Marker(list(df_seoul_ediya.iloc[i][['위도', '경도']]),
                 popup=df_seoul_ediya.iloc[i][['지점명']],
                 icon=folium.Icon(color='blue')).add_to(map_ediya)

map_ediya

- 서울시 스타벅스 및 이디야 입지 시각화

In [ ]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)

data_size = len(df_seoul_starbucks)
data2_size = len(df_seoul_ediya)

# 지도 정의
map = folium.Map(location=loc,
                 #tiles = 'Stamen Terrain',
                 #attr='Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL.',
                 zoom_start=11)

# 포인트 마커 추가

for i in range(data_size):

    folium.Marker(list(df_seoul_starbucks.iloc[i][['위도', '경도']]),
                 popup=df_seoul_starbucks.iloc[i][['지점명']],
                 icon=folium.Icon(color='green')).add_to(map)


for i in range(data2_size):

    folium.Marker(list(df_seoul_ediya.iloc[i][['위도', '경도']]),
                 popup=df_seoul_ediya.iloc[i][['지점명']],
                 icon=folium.Icon(color='blue')).add_to(map)


map

- 점포 별 반경 100m 서클 시각화

In [ ]:
# 위치 파라미터 설정

# 강남역 좌표
loc = [37.497895, 127.027565] # 위도(N), 경도(E)

data_size = len(df_seoul_starbucks)
data2_size = len(df_seoul_ediya)

# 지도 정의
map = folium.Map(location=loc, zoom_start=12)
folium.Choropleth(geo_data=geo_str,
              data = starbucks_gu['상호명'],
              columns=[starbucks_gu.index, starbucks_gu['상호명']],
              fill_color='PuRd',
              key_on='feature.id')

# 포인트 서클 추가

for i in range(data_size):

    folium.Circle(list(df_seoul_starbucks.iloc[i][['위도', '경도']]),
                  radius = 100,
                  popup = df_seoul_starbucks.iloc[i]['지점명'],
                  color = '#2c9147',fill_color = '#2c9147').add_to(map)

for i in range(data2_size):

    folium.Circle(list(df_seoul_ediya.iloc[i][['위도', '경도']]),
                  radius = 100,
                  popup = df_seoul_ediya.iloc[i]['지점명'],
                  color = '#32408c',fill_color = '#32408c').add_to(map)

map

- 결론   
점포 별 반경 100m 서클 시각화 결과는 강남역을 중심으로 확대한 지도를 보여주고 있습니다. 점포 마다 반경 100m 지역을 원으로 나타내고 있습니다. 반원을 점포의 직접적인 상권으로 가정하면, 초록색 원인 스타벅스와 파란색 원인 이디야 사이에 직접적인 상권이 겹치는 부분이 심심치 않게 보입니다. 또한, 두 브랜드 모두 특정 도로를 중심으로 분포하고 있는 모습을 확인할 수 있습니다. 시각화 결과만 놓고 보면 두 점포가 매우 가까이에 위치해 있는 것을 알 수 있습니다. 다만, 이는 두 브랜드를 비교한 결과이기때문에 스타벅스와 이디야만 근접해있다고 볼 수는 없다는 한계가 있습니다.

In [ ]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)

data_size = len(df_seoul_starbucks)
data2_size = len(df_seoul_ediya)

# 지도 정의
map = folium.Map(location=loc,
                 #tiles = 'Stamen Toner',
                 zoom_start=11)


# 지도 정의
map = folium.Map(location=loc, zoom_start=12)
folium.Choropleth(geo_data=geo_str,
              data = starbucks_gu['상호명'],
              columns=[starbucks_gu.index, starbucks_gu['상호명']],
              fill_color='PuRd',
              key_on='feature.id')

# 포인트 마커 추가

for i in range(data_size):

    folium.Marker(list(df_seoul_starbucks.iloc[i][['위도', '경도']]),
                 popup=df_seoul_starbucks.iloc[i][['지점명']],
                 icon=folium.Icon(color='green')).add_to(map)


for i in range(data2_size):

    folium.Marker(list(df_seoul_ediya.iloc[i][['위도', '경도']]),
                 popup=df_seoul_ediya.iloc[i][['지점명']],
                 icon=folium.Icon(color='blue')).add_to(map)


map